In [35]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import make_pipeline

import pickle

In [36]:
#Parameters
C = 1.0
n_splits = 5
output_file = f'model_C = {C}.bin'

In [37]:
# Data Prep

df = pd.read_csv("data.csv")
df.columns = df.columns.str.lower().str.replace(" ","_")

df["fbs"] = df["fbs"].map({True: 'true', False: 'false'})
df["exang"] = df["exang"].map({True: 'true', False: 'false'})

string = list(df.dtypes[df.dtypes == 'object'].index)

for c in string:
    df[c] = df[c].str.lower().str.replace(' ','_')
    
df["target"] = (df['num']>0).astype(int)

numerical = ['age',
 'trestbps',
 'chol',
 'thalch',
 'oldpeak',
 'ca'
 ]

categorical = [
 'sex',
 'cp',
 'fbs',
 'restecg',
 'exang',
 'slope',
 'thal',
] 

del df['num']
del df['id']
del df['dataset']

for n in numerical:
    df[n] = df[n].fillna(df[n].median())

for n in categorical:
    df[n] = df[n].fillna(df[n].mode()[0])

df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

In [38]:
#Training

def train(df_train, y_train, C= 0.1):
    #dv = DictVectorizer(sparse = False)
    train_dicts= df_train[categorical + numerical].to_dict(orient = 'records')

    pipeline = make_pipeline(
    DictVectorizer(sparse = False),
    LogisticRegression(C=C, solver = 'liblinear', max_iter=10000))
    

    
    #X_train = dv.fit_transform(train_dicts)

    #model = LogisticRegression(C=C, max_iter=10000)
    pipeline.fit(train_dicts, y_train)

    return pipeline

def predict(df, pipeline):
    dicts= df[categorical + numerical].to_dict(orient = 'records')
    #X = dv.transform(dicts)

    y_pred = pipeline.predict_proba(dicts)[:, 1]

    return y_pred

In [39]:
#Validation

print(f'doing validation with C = {C}')
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []
fold =0
for train_idx, val_idx  in  kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.target.values
    y_val = df_val.target.values

    pipeline  = train(df_train,  y_train, C=C)
    y_pred = predict(df_val, pipeline )

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)
    print(f'auc on fold {fold} is {auc}')
    fold = fold + 1
    
print('validation result:')
print(f'C={C} {np.mean(scores).round(3)}, {np.std(scores).round(3)}')


doing validation with C = 1.0
auc on fold 0 is 0.8420759962928638
auc on fold 1 is 0.879737335834897
auc on fold 2 is 0.8992926284437825
auc on fold 3 is 0.8829031051253272
auc on fold 4 is 0.8981757259865972
validation result:
C=1.0 0.88, 0.021


In [40]:
#Training the final model

pipeline  = train(df_full_train, df_full_train.target.values, C=0.1)
y_pred = predict(df_test, pipeline )

y_test = df_test.target.values
auc = roc_auc_score(y_test, y_pred)
print(f'auc = {auc}')

auc = 0.9142857142857144


In [98]:
#Save the model using Pickel

with open(output_file, 'wb') as f_out:
    pickle.dump((pipeline), f_out)

print(f'the model is saved to {output_file}')

In [96]:
output_file = f'model_C={C}.bin'
output_file

'model_C=10.bin'

In [97]:
'''f_out = open(output_file, 'wb')
pickle.dump((dv, model), f_out)
f_out.close()'''

"f_out = open(output_file, 'wb')\npickle.dump((dv, model), f_out)\nf_out.close()"

Load model

In [1]:
import pickle

In [2]:
model_file = 'model_C=10.bin'

In [3]:
with open(model_file, 'rb') as f_in:
    (dv,model) = pickle.load(f_in)

In [4]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(C=0.1, max_iter=100000))

In [5]:
patient = {
    'sex': 'Female',
    'cp': 'atypical angina',
    'fbs': 'false',
    'restecg': 'normal',
    'exang': 'false',
    'slope': 'upsloping',
    'thal': 'normal',
    'age': 71,
    'trestbps': 160.0,
    'chol': 302.0,
    'thalch': 162.0,
    'oldpeak': 0.4,
    'ca': 2.0
}

In [6]:
hasattr(dv, "vocabulary_"), hasattr(dv, "feature_names_")


(True, True)

In [7]:
X_patient = dv.transform([patient])
prob = model.predict_proba(X_patient)[0, 1]
prob

np.float64(0.14042874973981748)